In [1]:
import re
import json
import csv
import xml.etree.ElementTree as ET
import pandas as pd
import advertools as adv
import pathlib

# sample.log 파일 경로 설정 (이 스크립트가 실행되는 위치에 data/sample.log가 있다고 가정)
LOG_FILE_PATH = pathlib.Path("./data/sample.log")
OUTPUT_PARQUET_PATH = pathlib.Path("./data/parsed_logs.parquet") # 이 경로는 이제 advertools 직접 출력용으로 사용됩니다.
OUTPUT_ERROR_LOG_PATH = pathlib.Path("./data/error_logs.txt")
# advertools 출력을 위한 추가 파일 경로 설정
ADVERTOOLS_ERRORS_LOG_PATH = pathlib.Path("./data/advertools_error_logs.txt") # advertools 오류 로그 파일

# --- 1. 일반 텍스트 로그 파일 구문 분석 (sample.log에 집중) ---
# sample.log 파일이 존재하는지 확인
if not LOG_FILE_PATH.exists():
    print(f"오류: {LOG_FILE_PATH} 파일을 찾을 수 없습니다. 먼저 sample.log 파일을 생성해주세요.")
    print("이 스크립트를 실행하기 전에 이전 단계에서 sample.log를 생성했는지 확인하십시오.")
else:
    # 2.1 정규 표현식(Regex)을 이용한 구문 분석

    # Apache combined 로그 형식 정규 표현식
    # %h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-agent}i\"
    APACHE_COMBINED_REGEX = re.compile(
        r'^(?P<ip>\S+) (?P<identd>\S+) (?P<user>\S+) \[(?P<timestamp>[^\]]+)\] '
        r'"(?P<method>\S+) (?P<path>\S+) (?P<protocol>[^"]+)" (?P<status>\d+) (?P<size>\S+) '
        r'"(?P<referer>[^"]*)" "(?P<user_agent>[^"]*)"$'
    )

    # Nginx combined-like 로그 형식 정규 표현식
    # $remote_addr - $remote_user [$time_local] "$request" $status $body_bytes_sent
    # "$http_referer" "$http_user_agent" "$host" $request_time
    NGINX_COMBINED_REGEX = re.compile(
        r'^(?P<ip>\S+) - (?P<user>\S+) \[(?P<timestamp>[^\]]+)\] '
        r'"(?P<method>\S+) (?P<path>\S+) (?P<protocol>[^"]+)" (?P<status>\d+) (?P<size>\S+) '
        r'"(?P<referer>[^"]*)" "(?P<user_agent>[^"]*)" "(?P<host>[^"]*)" (?P<request_time>\S+)$'
    )

    parsed_lines = []
    unparsed_lines = []

    with LOG_FILE_PATH.open("r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            line = line.strip()
            if not line: # 빈 줄 건너뛰기
                continue

            # Apache 형식 먼저 시도
            match = APACHE_COMBINED_REGEX.match(line)
            if match:
                data = match.groupdict()
                data['log_format'] = 'apache_combined'
                parsed_lines.append(data)
            else:
                # Nginx 형식 시도
                match = NGINX_COMBINED_REGEX.match(line)
                if match:
                    data = match.groupdict()
                    data['log_format'] = 'nginx_combined'
                    parsed_lines.append(data)
                else:
                    unparsed_lines.append(line) # 어떤 형식에도 매치되지 않는 줄

    print("\n--- 정규 표현식 구문 분석 결과 (일부) ---")
    print(f"구문 분석된 로그 수: {len(parsed_lines)}")
    print(f"구문 분석 실패한 로그 수: {len(unparsed_lines)}")

    if parsed_lines:
        print("\n첫 5개 구문 분석된 로그 항목:")
        for log_entry in parsed_lines[:5]:
            print(log_entry)

    if unparsed_lines:
        print("\n첫 5개 구문 분석 실패한 로그 항목:")
        for unparsed_line in unparsed_lines[:5]:
            print(unparsed_line)
        # 구문 분석 오류 로그를 별도의 파일에 저장
        with OUTPUT_ERROR_LOG_PATH.open("w", encoding="utf-8") as ef:
            for error_line in unparsed_lines:
                ef.write(error_line + "\n")
        print(f"\n구문 분석 오류 로그가 '{OUTPUT_ERROR_LOG_PATH}'에 저장되었습니다.")

    print("-" * 60 + "\n")

    # 2.2 advertools 라이브러리를 이용한 구문 분석

    print("--- advertools 라이브러리를 이용한 구문 분석 ---")
    print("advertools는 'common' 및 'combined' 로그 형식을 기본 지원합니다.")
    print("이 라이브러리는 자동으로 데이터 유형을 변환하고, URL 및 사용자 에이전트를 구문 분석합니다.")

    try:
        # advertools를 사용하여 sample.log 구문 분석
        # combined 형식 지정 (Apache combined와 유사)
        # advertools 버전 0.13.0 이상에서는 logs_to_df()에 'output_file'과 'errors_file' 인자가 필수입니다.
        # output_file에 .parquet 확장자를 지정하여 advertools가 직접 parquet 파일로 저장하도록 지시합니다.
        adv.logs_to_df(
            log_file=str(LOG_FILE_PATH),
            log_format='combined', # 또는 custom_regex=[APACHE_COMBINED_REGEX, NGINX_COMBINED_REGEX]
            output_file=str(OUTPUT_PARQUET_PATH), # 구문 분석된 데이터를 저장할 파일 (Parquet)
            errors_file=str(ADVERTOOLS_ERRORS_LOG_PATH), # 오류 로그를 저장할 파일
        )

        # advertools가 직접 Parquet 파일로 저장했으므로, 이제 그 파일을 다시 읽어와서 head와 info를 출력합니다.
        print(f"\nAdvertools가 구문 분석된 로그를 '{OUTPUT_PARQUET_PATH}'에 직접 저장했습니다.")
        df_logs_adv = pd.read_parquet(OUTPUT_PARQUET_PATH) # 저장된 Parquet 파일을 다시 DataFrame으로 읽어옴

        print("\nAdvertools로 구문 분석된 DataFrame (첫 5줄):")
        print(df_logs_adv.head())

        print(f"\nDataFrame의 열 정보 (데이터 유형 및 null 값):")
        df_logs_adv.info()

        # 데이터 보강 예시: 사용자 에이전트 및 URL 필드 자동 구문 분석
        print("\nAdvertools에 의해 구문 분석된 사용자 에이전트 및 URL 필드 예시:")
        if 'device_type' in df_logs_adv.columns and 'os' in df_logs_adv.columns:
            print(f"첫 5개 장치 유형: {df_logs_adv['device_type'].head().tolist()}")
            print(f"첫 5개 OS: {df_logs_adv['os'].head().tolist()}")
        if 'url_path' in df_logs_adv.columns and 'url_query' in df_logs_adv.columns:
            print(f"첫 5개 URL 경로: {df_logs_adv['url_path'].head().tolist()}")
            print(f"첫 5개 URL 쿼리: {df_logs_adv['url_query'].head().tolist()}")


        # advertools가 이미 Parquet으로 저장했으므로, 이 부분은 더 이상 필요 없습니다.
        # df_logs_adv.to_parquet(OUTPUT_PARQUET_PATH, index=False, engine='pyarrow', compression='snappy')
        # print(f"\n구문 분석된 로그가 압축된 Parquet 형식으로 '{OUTPUT_PARQUET_PATH}'에 저장되었습니다.")

        # advertools가 반환한 오류 로그 확인 (errors_file 인자 사용 시)
        if ADVERTOOLS_ERRORS_LOG_PATH.exists() and ADVERTOOLS_ERRORS_LOG_PATH.stat().st_size > 0:
            print(f"\nadvertools에 의해 처리되지 않은 오류 로그가 '{ADVERTOOLS_ERRORS_LOG_PATH}'에 기록되었습니다.")
        else:
            print("\nadvertools에 의해 처리되지 않은 오류 로그 파일이 없거나 비어 있습니다.")

    except ImportError:
        print("\nadvertools 라이브러리가 설치되어 있지 않습니다. 'pip install advertools'로 설치해주세요.")
    except Exception as e:
        print(f"\nadvertools 구문 분석 중 오류 발생: {e}")



--- 정규 표현식 구문 분석 결과 (일부) ---
구문 분석된 로그 수: 603
구문 분석 실패한 로그 수: 16

첫 5개 구문 분석된 로그 항목:
{'ip': '52.79.152.10', 'identd': '-', 'user': '-', 'timestamp': '16/Aug/2025:09:46:08 +0000', 'method': 'POST', 'path': '/', 'protocol': 'HTTP/2.0', 'status': '301', 'size': '763', 'referer': 'https://www.bing.com/', 'user_agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 17_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Mobile/15E148 Safari/604.1', 'log_format': 'apache_combined'}
{'ip': '198.51.100.77', 'identd': '-', 'user': '-', 'timestamp': '16/Aug/2025:06:15:55 +0000', 'method': 'POST', 'path': '/login', 'protocol': 'HTTP/1.1', 'status': '401', 'size': '2642', 'referer': 'https://www.bing.com/', 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36', 'log_format': 'apache_combined'}
{'ip': '172.16.1.23', 'identd': '-', 'user': 'bob', 'timestamp': '17/Aug/2025:01:57:59 +0000', 'method': 'GET', 'path': '/ind